In [361]:
from gurobipy import *
import numpy as np
import math as mt

In [362]:
def random_matrice_utilite(n, p):
        return np.random.randint(50 ,size=(n,p))


In [363]:
def generate_w_prime(n):
        w=[i for i in range(n,0,-1)]
        w1=[]
        for i in range(n):
                if i==n-1 :
                        w1.append(w[i])
                else :
                        w1.append(w[i]-w[i+1])
        return w1

In [364]:
def generate_rk(n):
        coef_rk = []
        for k in range(n):
                for i in range(n):
                        coef_rk.append([])
                        for j in range(n):
                                coef_rk[-1].append(0)
                        coef_rk[-1][k] = 1
        coef_rk = np.array(coef_rk)
        return coef_rk

In [365]:
def generate_x(n, p, u):
        inf = 0
        cpt = 0
        coef_xi = []
        for i in range(n):
                coef_xi.append([0 for _ in range(n*p)])
                for j in range(inf, inf+p):
                        coef_xi[-1][j] = - u[i][j%p]
                inf += p
        res = coef_xi
        for i in range(n*n-n):
                res = np.vstack([res, coef_xi[i%n]])
        return res

In [366]:
def coef_cont_unicite(n, p):
        L2=np.zeros((p,1))
        for i in range(n):
            L1=np.identity(p)
        L2=np.concatenate((L1,L2),axis=1)
        L2=np.delete(L2,[-1],1)
        
        res = L2
        for i in range(n-1):
                res = np.concatenate((res, L2), axis=1)      
        zeros = np.zeros((p, n*n+n))
        return np.concatenate((zeros, res), axis=1)

coef_cont_unicite(3, 2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
        1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 1.]])

In [367]:
def matrice_contrainte(n, p, u):
        ## les variables rk
        coef_rk = generate_rk(n)
        
        ## les coef de bij 
        coef_bik = -1 * np.identity(n**2)
        
        # les coef de x
        coef_xi = generate_x(n, p, u)
        
        # contrainte sur l'unicite
        deux_personne_pas_le_mm = coef_cont_unicite(n, p)
        
        bik_xi = np.concatenate((coef_bik, coef_xi), axis=1)
        res = np.concatenate(
                (coef_rk, bik_xi), 
                axis=1
        )
        return np.vstack(
                (res, deux_personne_pas_le_mm)
        )

n, p = 3, 4
u = random_matrice_utilite(n, p)
matrice_contrainte(n, p, u)

array([[   1.,    0.,    0.,   -1.,   -0.,   -0.,   -0.,   -0.,   -0.,
          -0.,   -0.,   -0., -325., -225., -210., -115.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.],
       [   1.,    0.,    0.,   -0.,   -1.,   -0.,   -0.,   -0.,   -0.,
          -0.,   -0.,   -0.,    0.,    0.,    0.,    0., -325., -225.,
        -210., -115.,    0.,    0.,    0.,    0.],
       [   1.,    0.,    0.,   -0.,   -0.,   -1.,   -0.,   -0.,   -0.,
          -0.,   -0.,   -0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0., -325., -225., -210., -115.],
       [   0.,    1.,    0.,   -0.,   -0.,   -0.,   -1.,   -0.,   -0.,
          -0.,   -0.,   -0., -325., -225., -210., -115.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    1.,    0.,   -0.,   -0.,   -0.,   -0.,   -1.,   -0.,
          -0.,   -0.,   -0.,    0.,    0.,    0.,    0., -325., -225.,
        -210., -115.,    0.,    0.,    0.,    0.],
       [   0.,    1.,    0.,   -0.,

In [368]:
def second_membre(p):
        S= [0 for i in range (n*n)]
        for i in range(p):
                S.append(1)
        return S

In [369]:
def coef_func_objectif(n, p, w_prime):
        ## coefficient fonction objectif
        c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
        # coef des b_ik
        for i in range(n) :
                for j in range(n) :
                        c.append(-1*w_prime[i])

        c += [0 for i in range(n*p)] # ajout des coef de x
        c = np.array(c)
        #print(c)
        return c

In [370]:
# Creer les models de la fonction objectif
m = Model("Experimentation partie 2") 

In [371]:
def declaration_vars(n, p, m):
        # declaration des variables de decisions
        x = []
        for i in range(n):
                x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
        for i in range (n):
                for j in range(n):
                        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
        for i in range (n):
                for j in range(p):
                        x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d%d" % ((i+1,(j+1)))))

        # maj du modele pour integrer les nouvelles variables
        m.update()
        print(x)
        return x

In [372]:
def definition_objectif(n, p, a, x, m):
        # definition de l'objectif
        obj = LinExpr();
        obj =0
        for j in range(n * (n + p +1)):
                obj += a[j] * x[j]

        m.setObjective(obj,GRB.MAXIMIZE)
        return obj

In [373]:
def definition_contraintes(n, p, m, b, x, S):
        # Definition des contraintes
        lignes = range(n*n+p)
        colonnes = range(n*(n+p+1))

        for i in lignes:
                m.addConstr(quicksum(b[i][j]*x[j] for j in colonnes) <= S[i], "Contrainte%d" % i)

In [374]:
def display_vars(m):
        print("")                
        print('Solution optimale:')
        mvars = m.getVars()
        names = m.getAttr('VarName', mvars)
        values = m.getAttr('X', mvars)
        result = dict(zip(names, values))
        for key in result:
                print(key,'=',result[key])
        print("")
        print('Valeur de la fonction objectif :', m.objVal)


In [376]:
def experience(n1, p1):
        u = random_matrice_utilite(n1, p1)
        print(u)
        w1 = generate_w_prime(n1)
        print(w1)
        b = matrice_contrainte(n1, p1, u)
        print(b)
        S = second_membre(p1)
        print(S)
        a = coef_func_objectif(n1, p1, w1)
        # Creer les models de la fonction objectif
        m = Model("Experimentation partie 2") 
        x = declaration_vars(n1, p1, m)
        obj = definition_objectif(n1, p1, a, x, m)
        definition_contraintes(n1, p1, m, b, x, S)
        # Resolution
        m.optimize()
        display_vars(m)

experience(3, 6)

[[325.0, 225.0, 210.0, 115.0, 75.0, 50.0], [325.0, 225.0, 210.0, 115.0, 75.0, 50.0], [325.0, 225.0, 210.0, 115.0, 75.0, 50.0]]
[1, 1, 1]
[[   1.    0.    0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
  -325. -225. -210. -115.  -75.  -50.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   1.    0.    0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
     0.    0.    0.    0.    0.    0. -325. -225. -210. -115.  -75.  -50.
     0.    0.    0.    0.    0.    0.]
 [   1.    0.    0.   -0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  -325. -225. -210. -115.  -75.  -50.]
 [   0.    1.    0.   -0.   -0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.
  -325. -225. -210. -115.  -75.  -50.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    1.    0.   -0.   -0.   -0.   -0.   -1.   -0.   -0.   -0.   -0.
     0.    0.    0.    0.    0.    0. -32